# User Interface 

## Initilize and import

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import ipywidgets as widgets
import rospy
import time
import actionlib
from std_msgs.msg import String
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from ipywidgets import Button,GridBox,Layout,FloatSlider,ButtonStyle
import rt2_assignment1.msg

#init node
rospy.init_node('user_interface')

# velocities publisher
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
vel_pub = rospy.Publisher('velocities', Twist, queue_size=10)
msg = Twist()

# user interface client
ui_client = rospy.ServiceProxy('/user_interface', Command)

#action client
act_client = actionlib.SimpleActionClient('position',rt2_assignment1.msg.PositionAction)
if(act_client.wait_for_server()):
    print("Connected to Action Server")
else:
    print("Unable to connect to Action Server")

## Buttons to start and stop random target behavior

In [2]:
start_but = Button(description = "START", layout = Layout(grid_area='start_but'), style = ButtonStyle(button_color ='green'))
stop_but = Button(description = "STOP", layout = Layout(grid_area='stop_but'), style = ButtonStyle(button_color ='red'))

def on_start_clicked(b):
    global ui_client
    ui_client('start')
    print("start")

def on_stop_clicked(b):
    global ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    print("goal canceled")

start_but.on_click(on_start_clicked)
stop_but.on_click(on_stop_clicked)

# show buttons in a grid Box
gridBox = GridBox(children=[start_but, stop_but],
                         layout = Layout(
                            width = '50%',
                            grid_template_rows = '10%',
                            grid_template_columns = '50% 50%',
                            grid_template_areas = ''' "start_but stop_but" '''
                         )
                    )
display(gridBox)


GridBox(children=(Button(description='START', layout=Layout(grid_area='start_but'), style=ButtonStyle(button_c…

ServiceException: transport error completing service call: unable to receive data from sender, check sender's logs for details

## Slider to set linear and angular velocity

In [3]:
# slider for lin vel
lin = FloatSlider(min=0.0, max=2.0,value = 1.0)
display(lin)

#slider for ang vel
ang = FloatSlider(min=0.0, max=5.0, value = 3.0)
display(ang)

# fun to change lin vel ( maybe could send a twist on a topic and go to point could read it to set vel)
def on_value_change(change):
    global msg
    msg.linear.x = change['new']
    vel_pub.publish(msg)

# fun to change ang vel
def on_valueang_change(change):
    global msg
    msg.angular.z = change['new']
    vel_pub.publish(msg)

lin.observe(on_value_change, names='value')
ang.observe(on_valueang_change, names='value')

FloatSlider(value=1.0, max=2.0)

FloatSlider(value=3.0, max=5.0)

## Buttons to control the robot without random behavior

In [1]:
# declaration buttons
stop = Button(description = "STOP", layout= Layout(width='auto', grid_area='stop'))
forward = Button(description = "^", layout= Layout(width='auto', grid_area='forward'))
backward = Button(description = "v", layout= Layout(width='auto', grid_area='backward'))
right = Button(description = ">", layout= Layout(width='auto', grid_area='right'))
left = Button(description = "<", layout= Layout(width='auto', grid_area='left'))

# defining on_clicked functions
def on_stop_clicked(b):
    global msg,ui_client
    ui_client('stop')
    act_client.cancel_all_goals()
    msg.linear.x = 0
    msg.angular.z = 0
    pub.publish(msg)
    
def on_forward_clicked(b):
    global lin,msg,pub,ui_client
    ui_client('stop')
    act_client.cancel_all_goals()
    msg.linear.x = lin.value*0.3
    msg.angular.z = 0
    pub.publish(msg)
    
def on_backward_clicked(b):
    global lin,msg,pub,ui_client
    ui_client('stop')
    act_client.cancel_all_goals()
    msg.linear.x = -lin.value*0.3
    msg.angular.z = 0
    pub.publish(msg)
    
def on_left_clicked(b):
    global msg,ang,pub,ui_client
    ui_client('stop')
    act_client.cancel_all_goals()
    msg.linear.x = 0
    msg.angular.z = -ang.value*3.0
    pub.publish(msg)
    
def on_right_clicked(b):
    global msg,ang,pub,ui_client
    ui_client('stop')
    act_client.cancel_all_goals()
    msg.linear.x = 0
    msg.angular.z = ang.value*3.0
    pub.publish(msg)
    
# attach functions to buttons
stop.on_click(on_stop_clicked)
forward.on_click(on_forward_clicked)
backward.on_click(on_backward_clicked)
left.on_click(on_left_clicked)
right.on_click(on_right_clicked)

# show buttons in a grid Box
gridBox_buttons = GridBox(children=[stop, forward, backward, right, left],
                         layout = Layout(
                            width = '400px',
                            height = '100px',
                            grid_template_rows = '40% 40% 40%',
                            grid_template_columns = '20% 20% 20%',
                            grid_template_areas ='''
                            ". forward ."
                            "left stop right"
                            " . backward ."
                            '''
                         )
                    )
display(gridBox_buttons)

NameError: name 'Button' is not defined

## Line plot cmd_vel VS actual velocity

In [4]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import ipywidgets as widgets
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist

#set up the figure
fig_vel = plt.figure(1)

#set up axis
ax_lin = fig_vel.add_subplot(211)
ax_ang = fig_vel.add_subplot(212)

#lines
line_odom_lin, = ax_lin.plot([], [], label= "odom lin vel")
line_odom_ang, = ax_ang.plot([], [], label= "odom ang vel")
line_cmd_lin, = ax_lin.plot([],[], label= "cmd vel lin")
line_cmd_ang, = ax_ang.plot([],[], label= "cmd ang vel")

#axes features
ax_lin.grid()
ax_ang.grid()

ax_lin.legend()
ax_ang.legend()

ax_lin.set_title("linear velocities")
ax_ang.set_title("angular velocities")

ax_lin.set(xlabel = "Time", ylabel = "Magnitude")
ax_ang.set(xlabel = "Time", ylabel = "Magnitude")

# data
x_lin_data = []
x_ang_data = []

vlin_odom_data = []
vang_odom_data = []

vlin_cmd_data = []
vang_cmd_data = []

# initialize the plot
def init_vel():
    ax_lin.set_xlim (0,5) # time axis
    ax_ang.set_xlim (0,5) # time axis
    ax_lin.set_ylim(-0.8,0.8) #vel axis
    ax_ang.set_ylim(-0.8,0.8) #vel axis
    return [line_odom_lin,line_odom_ang,line_cmd_lin,line_cmd_ang],

# callbacks
def odom_callback(msg_odom):
    global vlin_odom_data, vang_odom_data, x_lin_data
    
    vlin_odom_data.append(msg_odom.twist.twist.linear.x)
    vang_odom_data.append(msg_odom.twist.twist.angular.z)
    
     # set x data on ax_lin 
    if(len(x_lin_data) != 0):
        x_lin_data.append(x_lin_data[len(x_lin_data)-1]+0.05)
    else:
        x_lin_data.append(0.05)
        
def cmd_vel_callback(msg_cmd):
    global vlin_cmd_data,vang_cmd_data,x_ang_data
    
    vlin_cmd_data.append(msg_cmd.linear.x)
    vang_cmd_data.append(msg_cmd.angular.z)
    
    # set x data on ax_ang
    if(len(x_ang_data) != 0):
        x_ang_data.append(x_ang_data[len(x_ang_data)-1]+0.05)
    else:
        x_ang_data.append(0.05)

def animate_vel(frame):   
    
    #update x axis ax_lin
    ax_lin_min,ax_lin_max = ax_lin.get_xlim()
     
    if(len(x_lin_data) != 0 and ax_lin_max < x_lin_data[len(x_lin_data)-1]):
        ax_lin.set_xlim(ax_lin_min, 2*ax_lin_max)
        ax_lin.figure.canvas.draw()
        
    #update x axis ax_ang
    ax_ang_min,ax_ang_max = ax_ang.get_xlim()
    
    if(len(x_ang_data) != 0 and ax_ang_max < x_ang_data[len(x_ang_data)-1]):
        ax_ang.set_xlim(ax_ang_min, 2*ax_ang_max)
        ax_ang.figure.canvas.draw()
        
    #update odom data
    line_odom_lin.set_data(x_lin_data,vlin_odom_data)
    line_odom_ang.set_data(x_ang_data,vang_odom_data)
    
    #update cmd data
    line_cmd_lin.set_data(x_lin_data,vlin_cmd_data)
    line_cmd_ang.set_data(x_ang_data,vang_cmd_data)    
    
    return [line_odom_lin,line_odom_ang,line_cmd_lin,line_cmd_ang],
    

#subscribers
sub_odom = jr.subscribe('/odom', Odometry, odom_callback)
sub_cmd_vel = jr.subscribe('/cmd_vel', Twist, cmd_vel_callback)  

# show the plots
anim = animation.FuncAnimation(fig_vel, animate_vel, init_func = init_vel)
fig_vel.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ERROR:tornado.application:Exception in callback <bound method TimerBase._on_timer of <matplotlib.backends.backend_webagg_core.TimerTornado object at 0x7ff3e8060040>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/usr/lib/python3/dist-packages/matplotlib/backend_bases.py", line 1194, in _on_timer
    ret = func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1447, in _step
    still_going = Animation._step(self, *args)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1173, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1192, in _draw_next_frame
    self._draw_frame(framedata)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1755, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipy

## Number of reached and canceled target

In [4]:
%matplotlib widget
num_reached = 0
num_canceled = 0
start_time = rospy.Time()
arrival_time = rospy.Time()
time_to_reach = []

def action_status_CB(msg):
    global num_reached, num_canceled, start_time, arrival_time, time_to_reach
    
    print("reade: " + msg.data)
    
    if(msg.data == "SUCCEEDED"):
        num_reached += 1
        print("num_reached: " + str(num_reached))
        #take arrival time
        arrival_time = rospy.get_rostime()
        time_to_reach.append(arrival_time.to_sec() - start_time.to_sec())
        print("time to reach a goal: "+ str(time_to_reach))
        
    elif(msg.data == "PREEMPTED"):
        num_canceled += 1  
        print("num_canceled: " + str(num_canceled))
        
    elif(msg.data == "PENDING"):
        #take starting time
        start_time = rospy.get_rostime()

# status subscriber
sub_status = jr.subscribe('action_status', String, action_status_CB)


#fig_targets, ax_targets = plt.subplots()
#bars = [0, 0]
#labels = ["Targets reached","Target canceled"]
#bar_plot = plt.bar(labels,bars)

#def init():
#    bar_plot.set_data(labels,bars)
 #   return bar_plot
    
#def animate_targets(i):
 #   global num_reached, num_canceled
  #  bars[0] = num_reached
   # bars[1] = num_canceled
   # bar_plot.set_data(labels,bars)
   # return bar_plot,
    
# show the plot
#anim = animation.FuncAnimation(fig_targets, animate_targets, init_func = init, blit = True)
   

## time required to reach the targets

In [ ]:
global time_to_reach
for i in range(len(time_to_reach)):
    print("time to reach goal " + i + ": " + time_to_reach[i])

## Show robot position and orientation

In [5]:
%matplotlib widget
# First set up the figure and the axis 
fig_odom, ax_odom = plt.subplots()
ax_odom.grid()

line, = plt.plot([], [], 'ro')
x_data = [] 
y_data = []

def init_odom():
    ax_odom.set_xlim(-5,5)
    ax_odom.set_ylim(-5,5)
    return line,

def odom_callback(msg):
    x_data.append(msg.pose.pose.position.x)
    y_data.append(msg.pose.pose.position.y)

def animate_odom(frame):
    line.set_data(x_data,y_data)
    return line,

sub_pos = jr.subscribe('odom', Odometry, odom_callback)

ani = animation.FuncAnimation(fig_odom, animate_odom, init_func = init_odom)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …